In [1]:
import pymysql
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
from flask import Flask, request, jsonify
import base64
from flask_cors import CORS
from io import BytesIO
import os

# Flask 애플리케이션 생성
app = Flask(__name__)
CORS(app)

# 연결에 필요한 정보
rds_host = 'capstone-database.c5ys4ks8sbyz.us-west-2.rds.amazonaws.com'
rds_port = 3306
rds_user = 'admin'  
rds_password = 'capstone'  
rds_database = 'capstone'  

# MySQL 연결
conn = pymysql.connect(
    host=rds_host,
    port=rds_port,
    user=rds_user,
    password=rds_password,
    database=rds_database
)


city_images_table = {
    "Corvallis": [],
    "Salem": [],
    "Portland": [],
    "Eugene": [],
    "Bend": [],
    "Beaverton": [],
    "Hillsboro": [],
    "Gresham": [],
    "Lake Oswego": [],
    "Tigard": [],
    "Grants Pass": [],
    "Oregon City": [],
    "Roseburg": [],
    "Hood River": []
}

city_list = [
    "Corvallis",
    "Salem",
    "Portland",
    "Eugene",
    "Bend",
    "Beaverton",
    "Hillsboro",
    "Gresham",
    "Lake Oswego",
    "Tigard",
    "Grants Pass",
    "Oregon City",
    "Roseburg",
    "Hood River"
]

# output_dir을 프로젝트 루트 디렉토리에 있는 'output' 폴더로 설정
output_dir = "output"

# output 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def get_db_connection():
    return pymysql.connect(
        host=rds_host,
        port=rds_port,
        user=rds_user,
        password=rds_password,
        database=rds_database,
        cursorclass=pymysql.cursors.DictCursor
    )

In [2]:
def merged_data(data1, data2, graph_type):
    type = 4 if graph_type == "humidity" else 2 if graph_type == "temp_max" else 3 if graph_type == "pressure" else 5 if graph_type == "temp_min" else None
    if type is None:
        raise ValueError("Invalid graph type")
    
    data = []
    for item1, item2 in zip(data1, data2):
        # 시간은 인덱스 0에 있으므로 별도로 처리
        time1, time2 = item1[0], item2[0]
        
        # 인덱스 type에 해당하는 값을 선택
        value1 = item1[type]
        value2 = item2[type]
        
        # 시간과 값들을 merged_data에 추가
        data.append((time1, value1, value2))

    return data

In [3]:
def compare_humidity_graph(data1, data2, city_name1, city_name2):
    data = merged_data(data1, data2, "humidity")
    #print("==data", data)
    df = pd.DataFrame(data, columns=["Time", "Humidity1", "Humidity2"])
    
    
    # x 축과 y 축 데이터 추출
    times = pd.to_datetime(df["Time"])
    humidity1 = df["Humidity1"]
    humidity2 = df["Humidity2"]
    
    # 그래프 생성
    plt.figure(figsize=(10, 6))
    plt.plot(times, humidity1, label= f"{city_name1} humidity", color="blue")
    plt.plot(times, humidity2, label= f"{city_name2} humidity", color="red")

    # 그래프 설정
    plt.xlabel("Time")
    plt.ylabel("humidity")
    plt.title("humidity change")
    plt.grid(True)
    plt.legend()  # 범례 표시

    
    # 이미지 파일로 저장
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    buffer.seek(0)

    #plt.show()
    # 그래프 창 닫기
    plt.close()

    # 이미지 파일을 base64로 인코딩하여 반환
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return encoded_image

def compare_temp_max_graph(data1, data2, city_name1, city_name2):
    data = merged_data(data1, data2, "temp_max")
    df = pd.DataFrame(data, columns=["Time", "Temp_max1", "Temp_max2"])
    
    # x 축과 y 축 데이터 추출
    times = pd.to_datetime(df["Time"])
    temp_max1 = df["Temp_max1"]
    temp_max2 = df["Temp_max2"]
    
    # 그래프 생성
    plt.figure(figsize=(10, 6))
    plt.plot(times, temp_max1, label= f"{city_name1} temp_max", color="blue")
    plt.plot(times, temp_max2, label= f"{city_name2} temp_max", color="red")

    # 그래프 설정
    plt.xlabel("Time")
    plt.ylabel("Max Temp")
    plt.title("Max Temperature change")
    plt.grid(True)
    plt.legend()  # 범례 표시

    
    # 이미지 파일로 저장
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    buffer.seek(0)
    #plt.show()
    # 그래프 창 닫기
    plt.close()

    # 이미지 파일을 base64로 인코딩하여 반환
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return encoded_image

def compare_temp_min_graph(data1, data2, city_name1, city_name2):
    data = merged_data(data1, data2, "temp_min")
    df = pd.DataFrame(data, columns=["Time", "Temp_min1", "Temp_min2"])
    
    # x 축과 y 축 데이터 추출
    times = pd.to_datetime(df["Time"])
    temp_min1 = df["Temp_min1"]
    temp_min2 = df["Temp_min2"]
    
    # 그래프 생성
    plt.figure(figsize=(10, 6))
    plt.plot(times, temp_min1, label= f"{city_name1} temp_max", color="blue")
    plt.plot(times, temp_min2, label= f"{city_name2} temp_max", color="red")

    # 그래프 설정
    plt.xlabel("Time")
    plt.ylabel("Min temp")
    plt.title("Min Temperature change")
    plt.grid(True)
    plt.legend()  # 범례 표시

    
    # 이미지 파일로 저장
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    buffer.seek(0)
    #plt.show()
    # 그래프 창 닫기
    plt.close()

    # 이미지 파일을 base64로 인코딩하여 반환
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return encoded_image


def compare_pressure_graph(data1, data2, city_name1, city_name2):
    data = merged_data(data1, data2, "pressure")
    df = pd.DataFrame(data, columns=["Time", "Pressure1", "Pressure2"])
    
    # x 축과 y 축 데이터 추출
    times = pd.to_datetime(df["Time"])
    pressure1 = df["Pressure1"]
    pressure2 = df["Pressure2"]
    
    # 그래프 생성
    plt.figure(figsize=(10, 6))
    plt.plot(times, pressure1, label= f"{city_name1} pressure", color="blue")
    plt.plot(times, pressure2, label= f"{city_name2} pressure", color="red")

    # 그래프 설정
    plt.xlabel("Time")
    plt.ylabel("pressure")
    plt.title("Pressure change")
    plt.grid(True)
    plt.legend()  # 범례 표시

    
    # 이미지 파일로 저장
    buffer = BytesIO()
    plt.savefig(buffer, format='png', transparent=True)
    buffer.seek(0)

    #plt.show()
    # 그래프 창 닫기
    plt.close()

    # 이미지 파일을 base64로 인코딩하여 반환
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return encoded_image


In [4]:
@app.route('/drawtable', methods=['POST'])
def send_table_name():
    content = request.get_json()
    city_name1 = content["city_name1"]
    city_name2 = content["city_name2"]

    conn = get_db_connection()
    if conn is None:
        return jsonify({"error": "Could not connect to the database"}), 500

    try:
        with conn.cursor() as cursor:
            query1 = f"SELECT time, date, pop, maximum_temperature, pressure, humidity, description, lowest_temperature FROM {city_name1} WHERE date BETWEEN CURDATE() AND DATE_ADD(CURDATE(), INTERVAL 5 DAY);"
            cursor.execute(query1)
            result1 = cursor.fetchall()

            query2 = f"SELECT time, date, pop, maximum_temperature, pressure, humidity, description, lowest_temperature FROM {city_name2} WHERE date BETWEEN CURDATE() AND DATE_ADD(CURDATE(), INTERVAL 5 DAY);"
            cursor.execute(query2)
            result2 = cursor.fetchall()

            formatted_data1 = process_data(result1)
            formatted_data2 = process_data(result2)

            max_temp_img = compare_temp_max_graph(formatted_data1, formatted_data2, city_name1, city_name2)
            min_temp_img = compare_temp_min_graph(formatted_data1, formatted_data2, city_name1, city_name2)
            humidity_img = compare_humidity_graph(formatted_data1, formatted_data2, city_name1, city_name2)
            pressure_img = compare_pressure_graph(formatted_data1, formatted_data2, city_name1, city_name2)

            query = "INSERT INTO capstone.images (max_temp_img, min_temp_img, humidity_img, pressure_img) VALUES (%s, %s, %s, %s)"
            cursor.execute(query, (max_temp_img, min_temp_img, humidity_img, pressure_img))
            conn.commit()
            image_id = cursor.lastrowid
            print("Inserted image id:", image_id)

            return jsonify({"id": image_id})
    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
        conn.rollback()
        return jsonify({"error": str(e)}), 500
    finally:
        conn.close()

def process_data(result):
    formatted_data = []
    for item in result:
        # 시간과 날짜 추출
        '''
        time = item[0]  # timedelta
        date = item[1]  # date
        pop = item[2] 
        max_temp = item[3] # temperture
        pressure = item[4] # pressure
        humidity = item[5] # humidity
        description = item[6] #description
        min_temp = item[7]
        '''
        # 시간과 날짜 추출
        time = item['time']  # timedelta
        date = item['date']  # date
        pop = item['pop']
        max_temp = item['maximum_temperature']  # temperature
        pressure = item['pressure']  # pressure
        humidity = item['humidity']  # humidity
        description = item['description']  # description
        min_temp = item['lowest_temperature']
        
        # datetime 객체 생성
        datetime_obj = datetime.combine(date, datetime.min.time()) + time
        
        # 데이터 포맷 변경
        formatted_data.append((datetime_obj, pop, max_temp, pressure, humidity, min_temp))

    # 데이터 정렬
    formatted_data = sorted(formatted_data, key=lambda x: x[0])
    return formatted_data


In [5]:

# 업데이트 코드
@app.route('/drawprecip', methods=['GET'])
def precipitation_graph():
    # 쿼리 실행
    for city in city_list:
        city_name = f"{city}"
        table_name = f"{city}_images"
        with conn.cursor() as cursor:
            # 첫 번째 도시에 대한 쿼리 실행
            query = f"SELECT time, date, pop, maximum_temperature, description, humidity FROM `{city_name}`;"
            cursor.execute(query)
            result = cursor.fetchall()
            formatted_data = []
            for item in result:
                # 시간과 날짜 추출
                time = item[0]  # timedelta
                date = item[1]  # date
                pop = item[2] 
                max_temp = item[3] # temperture
                description = item[4] #description
                humidity = item[5]
                
                datetime_obj = datetime.combine(date, datetime.min.time()) + time
                formatted_data.append((datetime_obj, pop, max_temp, description, humidity))
    
            ids = draw_precip_graph(formatted_data, cursor, table_name)
            city_images_table[city_name] = ids
    return jsonify(city_images_table)


def draw_precip_graph(data, cursor, table_name):
    df = pd.DataFrame(data, columns=["time", "precip", "temperature", "description", "humidity"])
    # 날짜 및 시간 형식 변환
    df["time"] = pd.to_datetime(df["time"])
    df["temperature"] = df["temperature"].apply(kelvin_to_fahrenheit)
    
    # 날짜별로 데이터 그룹화
    grouped_data = df.groupby(df["time"].dt.date)
    ids = []

    # 각 날짜별로 그래프 그리기
    for i, (date, group) in enumerate(grouped_data): 
        plt.figure(figsize=(10, 4))
        plt.rcParams['lines.linewidth'] = 2.0
    
        # 그래프 배경 투명하게 설정
        plt.gca().set_facecolor('none')
    
        # x축 y축 배경 삭제
        plt.box(False)
    
        # y축 배경 라인, 눈금, 라벨 삭제
        plt.tick_params(axis='y', which='both', left=False, right=False, labelleft=True, top=False, bottom=False)
    
        # 눈금 간격 조정 (x축만)
        plt.locator_params(axis='x', nbins=len(group["time"]))  # 눈금 개수를 그룹의 시간 개수와 동일하게 설정
        plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=True)  # x축 눈금 라인 및 라벨 유지
        #plt.xticks([])
        
        plt.plot(group["time"], group["precip"], marker='o', linestyle='-', alpha=0.5, linewidth=3)
    
        # 지정하지 않으면 삭제됨
        plt.ylabel("Precipitation")
        #plt.xlabel("Time")
        plt.grid(True)
        # y축 범위를 0 이상으로 설정
        plt.ylim(0, None)
    
        # 눈금 라인 삭제 (모두)
        plt.tick_params(axis='both', length=0)

        # 파일로 저장
        #filename = os.path.join(output_dir, f"graph_{i}.png")
       # plt.savefig(filename, format='png', transparent=True)

        # 이미지 파일로 저장
        buffer = BytesIO()
        plt.savefig(buffer, format='png', transparent=True)
        buffer.seek(0)

        #plt.show()
        
        # 그래프 창 닫기
        plt.close()
        
        precip_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
        #print(precip_image)

        '''temp 이미지 ''' 
        plt.figure(figsize=(10, 4))
        plt.rcParams['lines.linewidth'] = 2.0
    
        # 그래프 배경 투명하게 설정
        plt.gca().set_facecolor('none')
    
        # x축 y축 배경 삭제
        plt.box(False)
    
        # y축 배경 라인, 눈금, 라벨 삭제
        plt.tick_params(axis='y', which='both', left=True, right=False, labelleft=True, top=False, bottom=False)
    
        # 눈금 간격 조정 (x축만)
        plt.locator_params(axis='x', nbins=len(group["time"]))  # 눈금 개수를 그룹의 시간 개수와 동일하게 설정
        plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=True)  # x축 눈금 라인 및 라벨 유지
        #plt.xticks([])
        
        plt.plot(group["time"], group["temperature"], marker='o', linestyle='-', alpha=0.5, linewidth=3, color = 'yellow')
    
        # 지정하지 않으면 삭제됨
        plt.ylabel("Temperature (°F)")
        #plt.xlabel("Time")
        plt.grid(True)
        # y축 범위를 0 이상으로 설정
        plt.ylim(0, None)
    
        # 눈금 라인 삭제 (모두)
        plt.tick_params(axis='both', length=0)

        # 파일로 저장
        #filename = os.path.join(output_dir, f"graph_{i}.png")
        #plt.savefig(filename, format='png', transparent=True)

        #이미지 파일로 저장
        buffer = BytesIO()
        plt.savefig(buffer, format='png', transparent=True)
        buffer.seek(0)

        #plt.show()
        
        # 그래프 창 닫기
        plt.close()

        temp_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
        date = date
        precip = group["precip"].iloc[0]
        humidity = group["humidity"].iloc[0]
        description = group["description"].iloc[0]
        temp = group["temperature"].iloc[0]

        print("table_name2: ", table_name)
    
        # 테이블 이름을 백틱으로 감싸서 쿼리 작성
        query = f"INSERT INTO `{table_name}` (date, precipitation, temperature, description, humidity, image_data1, image_data2) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(query, (date, precip, temp, description, humidity, precip_image, temp_image))
        conn.commit()
    
        ids.append(cursor.lastrowid)
        
    return ids

        

def kelvin_to_fahrenheit(kelvin):
    return round((kelvin - 273.15) * 9/5 + 32, 2)

In [ ]:
@app.route('/mao')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
     app.run(host='0.0.0.0', port=8000)  # 포트는 원하는 포트로 변경 가능